In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from favorita import functions
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning, )
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

/home/emilien/.pyenv/versions/favorita/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


# Get the best params from the Grid Search

In [2]:
df_params = pd.read_csv('/home/emilien/code/EmilienStoret/favorita/Data/best_params_dataframe.csv')
df_best_params = df_params[df_params['mdape']==df_params['mdape'].min()]
df_best_params

,Unnamed: 0,changepoint_prior_scale,fourier_order,prior_scale_dcoilwtico,prior_scale_is_holiday,prior_scale_onpromotion,seasonality_mode,seasonality_prior_scale,mdape
263,263,0.08,7,0.01,0.1,0.01,multiplicative,0.11,0.117857


In [3]:
index = df_best_params.index[0]
index

263

# Get the train and test datasets for all the stores

In [4]:
df_main = pd.read_csv('/home/emilien/code/EmilienStoret/favorita/Data/df_main_V4.csv')
df_main.tail()

,id,date,store_nbr,family,sales,onpromotion,city,holidays,holidays_nat,is_holiday,dcoilwtico
3008275,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,0,0,0,47.57
3008276,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,0,0,0,47.57
3008277,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,0,0,0,47.57
3008278,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,0,0,0,47.57
3008279,3000887,2017-08-15,9,SEAFOOD,16.000,0,Quito,0,0,0,47.57


In [5]:
df_main_stores = df_main.groupby(['date','store_nbr']).agg({'sales':'sum',
                                                         'onpromotion':'sum',
                                                         'is_holiday':'mean',
                                                         'dcoilwtico':'mean'})
df_main_stores.reset_index(inplace=True)

for i in range(1, df_main_stores['store_nbr'].max()+1):
    df_loop=df_main_stores[df_main_stores['store_nbr']==i]
    df_loop.reset_index(inplace=True, drop=True)
    df_loop=df_loop.rename(columns={'date':'ds', 'sales':'y'})
    df_loop.drop(columns='store_nbr', inplace=True)
    globals()['df_store_'+str(i)] = df_loop
    globals()['df_train_store_'+str(i)]=df_loop[df_loop['ds']<'2017-04-01']
    globals()['df_test_store_'+str(i)]=df_loop[(df_loop['ds']>='2017-04-01')&(df_loop['ds']<'2017-07-01')]
    globals()['df_test_store_'+str(i)].reset_index(inplace=True, drop=True)
    
df_store_54.head()

,ds,y,onpromotion,is_holiday,dcoilwtico
0,2013-01-01,0.000,0,1.0,93.14
1,2013-01-02,4973.285,0,0.0,93.14
2,2013-01-03,3901.570,0,0.0,92.97
3,2013-01-04,3266.966,0,0.0,93.12
4,2013-01-05,4394.549,0,1.0,93.12


# Drop the outliers

In [6]:
for i in range(1, df_main_stores['store_nbr'].max()+1):
    q1, q3 = np.percentile(globals()['df_train_store_'+str(i)]['y'],[25,75])
    IQR = q3-q1
    df_outliers = globals()['df_train_store_'+str(i)][globals()['df_train_store_'+str(i)]['y']>3*IQR]
    globals()['df_train_store_'+str(i)].drop(index=df_outliers.index, inplace=True)

/tmp/ipykernel_9581/1933209214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_train_store_'+str(i)].drop(index=df_outliers.index, inplace=True)


# Train prophet

In [7]:
from tqdm import tqdm

In [8]:
for i in tqdm(range(1, df_main_stores['store_nbr'].max()+1)):
    
    # Train the model
    df_train = globals()['df_train_store_'+str(i)]

    m = Prophet(changepoint_prior_scale = df_best_params['changepoint_prior_scale'][index],
                seasonality_prior_scale = df_best_params['seasonality_prior_scale'][index],
                seasonality_mode = df_best_params['seasonality_mode'][index],
                weekly_seasonality=True,
                daily_seasonality = False,
                yearly_seasonality = True,
                interval_width=0.95)
    m.add_regressor('is_holiday', prior_scale = df_best_params['prior_scale_is_holiday'][index])
    m.add_regressor('onpromotion', prior_scale = df_best_params['prior_scale_onpromotion'][index])
    m.add_regressor('dcoilwtico', prior_scale = df_best_params['prior_scale_dcoilwtico'][index])
    m.add_seasonality(name='monthly', period=30.417, fourier_order=df_best_params['fourier_order'][index])
    model=m.fit(df_train[['ds', 'y', 'is_holiday', 'onpromotion', 'dcoilwtico']])

    #forecast from prophet
    df_loop = globals()['df_store_'+str(i)]
    future=m.make_future_dataframe(periods=91, freq= 'D')
    future.insert(1, 'is_holiday', df_loop['is_holiday'])
    future.insert(2, 'onpromotion', df_loop['onpromotion'])
    future.insert(3, 'dcoilwtico', df_loop['dcoilwtico'])
    globals()['df_store_'+str(i)+'_pred'] = m.predict(future)
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:36<00:00,  1.49it/s]


In [9]:
df_store_2_pred.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,dcoilwtico,dcoilwtico_lower,dcoilwtico_upper,extra_regressors_multiplicative,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2013-01-01,6962.182990,3538.675394,10262.308324,6962.182990,6962.182990,-0.000334,-0.000334,-0.000334,-0.025075,...,-0.094028,-0.094028,-0.094028,0.048545,0.048545,0.048545,0.0,0.0,0.0,6865.933496
1,2013-01-02,6969.735276,3869.115426,10790.404316,6969.735276,6969.735276,-0.000334,-0.000334,-0.000334,-0.034693,...,-0.040020,-0.040020,-0.040020,0.043243,0.043243,0.043243,0.0,0.0,0.0,7171.571925
2,2013-01-03,6977.287563,2810.953273,9516.009708,6977.287563,6977.287563,-0.000331,-0.000331,-0.000331,-0.034690,...,-0.171214,-0.171214,-0.171214,0.038990,0.038990,0.038990,0.0,0.0,0.0,6310.398398
3,2013-01-04,6984.839849,3613.562878,10233.893336,6984.839849,6984.839849,-0.000334,-0.000334,-0.000334,-0.034693,...,-0.060111,-0.060111,-0.060111,0.035839,0.035839,0.035839,0.0,0.0,0.0,7062.628135
4,2013-01-05,6992.392136,5526.210480,11991.741817,6992.392136,6992.392136,-0.000334,-0.000334,-0.000334,-0.025075,...,0.210391,0.210391,0.210391,0.033809,0.033809,0.033809,0.0,0.0,0.0,8826.531783


In [10]:
# Get MDAPE on Q2 2017

d_score={'store_nbr':[], 'MAPE': []}

for i in range(1, df_main_stores['store_nbr'].max()+1):
    df_loop = globals()['df_store_'+str(i)+'_pred']
    globals()['df_store_'+str(i)+'_pred_only']=df_loop[(df_loop['ds']>='2017-04-01')&(df_loop['ds']<'2017-07-01')]
    globals()['df_store_'+str(i)+'_pred_only'].reset_index(inplace = True, drop =True)
    mdape = abs((globals()['df_store_'+str(i)+'_pred_only']['yhat']-globals()['df_test_store_'+str(i)]['y'])
                /globals()['df_test_store_'+str(i)]['y'])
    d_score['store_nbr'].append(i)
    d_score['MAPE'].append(mdape.mean())
    
d_score = pd.DataFrame(d_score)


In [11]:
d_score

,store_nbr,MAPE
0,1,0.141432
1,2,0.101343
2,3,0.122630
3,4,0.101159
4,5,0.173313
5,6,0.105585
6,7,0.086867
7,8,0.123862
8,9,0.128801
9,10,0.182793


In [19]:
changepoint_prior_scale=df_best_params['changepoint_prior_scale'][index]
seasonality_prior_scale=df_best_params['seasonality_prior_scale'][index]
seasonality_mode=df_best_params['seasonality_mode'][index]
prior_scale_is_holiday=df_best_params['prior_scale_is_holiday'][index]
prior_scale_is_onpromotion=df_best_params['prior_scale_onpromotion'][index]
prior_scale_dcoilwtico=df_best_params['prior_scale_dcoilwtico'][index]
fourier_order=df_best_params['fourier_order'][index]

In [20]:
def get_forecast(df_train,
                df_regressor,
                changepoint_prior_scale, 
                seasonality_prior_scale, 
                seasonality_mode, 
                prior_scale_is_holiday, 
                prior_scale_is_onpromotion,
                prior_scale_dcoilwtico,
                fourier_order):
    
    m = Prophet(changepoint_prior_scale = changepoint_prior_scale,
                seasonality_prior_scale = seasonality_prior_scale,
                seasonality_mode = seasonality_mode,
                weekly_seasonality=True,
                daily_seasonality = False,
                yearly_seasonality = True,
                interval_width=0.95)
    m.add_regressor('is_holiday', prior_scale = prior_scale_is_holiday)
    m.add_regressor('onpromotion', prior_scale = prior_scale_is_onpromotion)
    m.add_regressor('dcoilwtico', prior_scale = prior_scale_dcoilwtico)
    m.add_seasonality(name='monthly', period=30.417, fourier_order=fourier_order)
    model=m.fit(df_train[['ds', 'y', 'is_holiday', 'onpromotion', 'dcoilwtico']])

    #forecast from prophet
    future=m.make_future_dataframe(periods=91, freq= 'D')
    future.insert(1, 'is_holiday', df_regressor['is_holiday'])
    future.insert(2, 'onpromotion', df_regressor['onpromotion'])
    future.insert(3, 'dcoilwtico', df_regressor['dcoilwtico'])
    
    return  m.predict(future)

In [22]:
get_forecast(df_train_store_1,
            df_store_1,
            changepoint_prior_scale, 
            seasonality_prior_scale, 
            seasonality_mode, 
            prior_scale_is_holiday, 
            prior_scale_is_onpromotion,
            prior_scale_dcoilwtico,
            fourier_order)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,dcoilwtico,dcoilwtico_lower,dcoilwtico_upper,extra_regressors_multiplicative,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2013-01-01,4848.393761,1588.036289,7566.314336,4848.393761,4848.393761,-0.003778,-0.003778,-0.003778,-0.195309,...,0.032312,0.032312,0.032312,0.070796,0.070796,0.070796,0.0,0.0,0.0,4467.650333
1,2013-01-02,4852.380267,2896.138505,9240.768764,4852.380267,4852.380267,-0.003778,-0.003778,-0.003778,-0.034051,...,0.223321,0.223321,0.223321,0.063039,0.063039,0.063039,0.0,0.0,0.0,6133.702664
2,2013-01-03,4856.366773,2178.861756,8077.997587,4856.366773,4856.366773,-0.003750,-0.003750,-0.003750,-0.034023,...,-0.002640,-0.002640,-0.002640,0.056179,0.056179,0.056179,0.0,0.0,0.0,5070.951990
3,2013-01-04,4860.353279,2456.299899,8591.182409,4860.353279,4860.353279,-0.003775,-0.003775,-0.003775,-0.034048,...,0.103052,0.103052,0.103052,0.050298,0.050298,0.050298,0.0,0.0,0.0,5616.876540
4,2013-01-05,4864.339785,1767.047150,7574.839333,4864.339785,4864.339785,-0.003775,-0.003775,-0.003775,-0.195306,...,0.078754,0.078754,0.078754,0.045446,0.045446,0.045446,0.0,0.0,0.0,4648.716701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,2017-06-26,10062.622303,8863.701910,15084.099248,9600.738223,10457.200468,0.004075,0.004075,0.004075,0.163493,...,0.063748,0.063748,0.063748,0.010183,0.010183,0.010183,0.0,0.0,0.0,11884.428549
1622,2017-06-27,10062.544108,7389.098604,13653.645459,9590.918447,10462.736225,0.004126,0.004126,0.004126,0.061738,...,0.032312,0.032312,0.032312,0.009459,0.009459,0.009459,0.0,0.0,0.0,10392.449783
1623,2017-06-28,10062.465913,9890.199897,15839.991392,9584.480926,10468.206065,0.004084,0.004084,0.004084,0.103898,...,0.223321,0.223321,0.223321,0.008975,0.008975,0.008975,0.0,0.0,0.0,12847.133554
1624,2017-06-29,10062.387718,7446.145226,13252.072745,9578.165796,10473.675904,0.004084,0.004084,0.004084,0.054300,...,-0.002640,-0.002640,-0.002640,0.008814,0.008814,0.008814,0.0,0.0,0.0,10431.326670


# Compare the results with the baseline

In [12]:
df_baseline_score = pd.read_csv('/home/emilien/code/EmilienStoret/favorita/Data/baseline_score.csv')
df_baseline_score.rename(columns={"store":"store_nbr"}, inplace=True)


In [13]:
d_score['store_nbr'].head()

0    1
1    2
2    3
3    4
4    5
Name: store_nbr, dtype: int64

In [14]:
df_baseline_score['store_nbr'].head()

0     1
1    10
2    11
3    12
4    13
Name: store_nbr, dtype: int64

In [15]:
df_score = d_score.merge(df_baseline_score, on='store_nbr', how='left')
df_score.drop(columns='Unnamed: 0', inplace = True)

In [16]:
df_score

,store_nbr,MAPE,baseline_score
0,1,0.141432,0.153758
1,2,0.101343,0.111695
2,3,0.122630,0.093054
3,4,0.101159,0.114829
4,5,0.173313,0.075818
5,6,0.105585,0.093011
6,7,0.086867,0.080799
7,8,0.123862,0.091296
8,9,0.128801,0.139706
9,10,0.182793,0.115569


In [17]:
#df_score.drop(columns='Unnamed: 0', inplace = True)
df_score['is_better']=df_score['MAPE']-df_score['baseline_score']
df_score['is_better']=df_score['is_better'].apply(lambda x: 1 if x<0 else 0)
df_score

,store_nbr,MAPE,baseline_score,is_better
0,1,0.141432,0.153758,1
1,2,0.101343,0.111695,1
2,3,0.122630,0.093054,0
3,4,0.101159,0.114829,1
4,5,0.173313,0.075818,0
5,6,0.105585,0.093011,0
6,7,0.086867,0.080799,0
7,8,0.123862,0.091296,0
8,9,0.128801,0.139706,1
9,10,0.182793,0.115569,0


In [18]:
df_score['is_better'].sum()

15